# Set up

## Install

In [1]:
!nvidia-smi

Mon Feb 10 00:19:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  |   00000000:25:00.0 Off |                    0 |
| N/A   36C    P0             48W /  250W |       1MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
import os
import torch
import yaml
import shutil
import zipfile
import ultralytics
import random
import requests
import cv2
import itertools
import json

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch  # For PyTorch
print(torch.cuda.is_available())  # True means a GPU is accessible

from ultralytics import YOLO
from ultralytics import settings
ultralytics.checks()

Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
Setup complete ✅ (64 CPUs, 503.2 GB RAM, 0.0/0.0 GB disk)


In [3]:
import subprocess

def install_opencv_headless():
    try:
        subprocess.run(["pip", "install", "opencv-python-headless"], check=True)
        print("opencv-python-headless installed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

install_opencv_headless()

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
opencv-python-headless installed successfully.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


## Ultralytics settings

In [4]:
# refer to the following page
# https://docs.ultralytics.com/quickstart/#modifying-settings

In [5]:
!pwd

/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/code


In [6]:
base_dir = "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc"
base_dir

'/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc'

In [7]:
#print(settings)

In [8]:
# refer to the following page
# https://docs.ultralytics.com/quickstart/#modifying-settings

# Update a setting+
settings.update({"datasets_dir": os.path.join(base_dir, "cvat_export")})
settings.update({"weights_dir": os.path.join(base_dir, "current_training/weights")})
settings.update({"runs_dir": os.path.join(base_dir, "current_training/runs")})
print(settings)

# Reset settings to default values
# settings.reset()

JSONDict("/home/hice1/jlee4338/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export",
  "weights_dir": "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/weights",
  "runs_dir": "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs",
  "uuid": "03e9e516df57c4506ba2f666b65d3a494d7d5521dfb3553eec5e6c3ca7475924",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}


## Import dataset

In [9]:
os.getcwd()

'/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/code'

In [10]:
!ls ../../cvat_export

ls: cannot access '../../cvat_export': No such file or directory


In [11]:
base_dir = "../cvat_export"

In [12]:
# Path to the ZIP file
zip_file_path = os.path.join(base_dir, "cvat_0209_trainvaltest_processed.zip")
extract_to = os.path.join(base_dir, "cvat_0209_trainvaltest_processed_unzipped")
# Open the ZIP file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print(f"Files extracted to: {extract_to}")

Files extracted to: ../cvat_export/cvat_0209_trainvaltest_processed_unzipped


In [13]:
!ls ../cvat_export/cvat_0209_trainvaltest_processed_unzipped

data_nc_5.yaml	images	labels


In [12]:
directory = "../cvat_export/cvat_0209_trainvaltest_processed_unzipped"

for root, dirs, files in os.walk(directory):
    level = root.replace(directory, "").count(os.sep)
    indent = " " * (level * 4)
    num_files = len(files)
    print(f"{indent}{os.path.basename(root)}   {num_files}/")
    sub_indent = " " * ((level + 1) * 4)
    #for f in files:
        #print(f"{sub_indent}{f}")

cvat_0209_trainvaltest_processed_unzipped   1/
    images   0/
        trainval   0/
        train   1346/
        test   167/
        val   238/
    labels   3/
        trainval   0/
        train   1346/
        test   167/
        val   238/
    .ipynb_checkpoints   1/


### Change .yaml files if needed

In [16]:
# always start the path from the "datasets_dir": "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export"
# path: ./export_0122_0921_processed_unzipped
# train: ./images/train  
# val: ./images/val  

base_dir = "../cvat_export/cvat_0209_trainvaltest_processed_unzipped"

# Filepath to your YAML file
yaml_file = os.path.join(base_dir, "data_nc_5.yaml")

with open(yaml_file, "r") as file:
    data = yaml.safe_load(file)
    print(data)
    data["path"] = "./cvat_0209_trainvaltest_processed_unzipped"
    print(data["path"])

# Write the changes back to the YAML file
with open(yaml_file, "w") as file:
    yaml.dump(data, file, default_flow_style=False)

print("Updated .yaml path")

{'names': {0: 'prohibAnyT', 1: 'prohibSomeT', 2: 'prohibBusStop', 3: 'prohibClean', 4: 'allow'}, 'nc': 5, 'path': './cvat_0209_trainvaltest_processed_unzipped', 'test': './images/test', 'train': './images/train', 'val': './images/val'}
./cvat_0209_trainvaltest_processed_unzipped
Updated .yaml path


## Make a duplicate not containing ATL in trainval

In [17]:
base_dir = "../cvat_export"

In [18]:
src_dir = os.path.join(base_dir, "cvat_0209_trainvaltest_processed_unzipped")
dst_dir = os.path.join(base_dir, "cvat_0209_trainvaltest_processed_unzipped_wonyc")

shutil.copytree(src_dir, dst_dir, dirs_exist_ok=True)  # Set `dirs_exist_ok=True` to overwrite if needed

print(f"Directory copied from {src_dir} to {dst_dir}")

Directory copied from ../cvat_export/cvat_0209_trainvaltest_processed_unzipped to ../cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc


In [19]:
directory = "../cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc"

for root, dirs, files in os.walk(directory):
    level = root.replace(directory, "").count(os.sep)
    indent = " " * (level * 4)
    num_files = len(files)
    print(f"{indent}{os.path.basename(root)}   {num_files}/")
    sub_indent = " " * ((level + 1) * 4)
    #for f in files:
        #print(f"{sub_indent}{f}")

cvat_0209_trainvaltest_processed_unzipped_wonyc   1/
    images   0/
        trainval   0/
        train   1346/
        test   167/
        val   238/
    labels   3/
        trainval   0/
        train   1346/
        test   167/
        val   238/
    .ipynb_checkpoints   1/


In [21]:
import glob

# Define the base directory
base_dir = "../cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc"

# Define the subdirectories to search (train and val under images and labels)
sub_dirs = [
    "images/train", "images/val",
    "labels/train", "labels/val"
]

# Loop through each subdirectory and remove files containing "mutcd"
for sub_dir in sub_dirs:
    accum = 0
    full_path = os.path.join(base_dir, sub_dir)
    
    # Search for files containing "mutcd" in the filename
    files_to_remove = glob.glob(os.path.join(full_path, "*gsv*"))
    
    # Delete the files
    for file_path in files_to_remove:
        try:
            os.remove(file_path)
            #print(f"Deleted: {file_path}")
            accum = accum + 1
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

    print(f"{accum} deleted in {sub_dir}")

0 deleted in images/train
0 deleted in images/val
0 deleted in labels/train
0 deleted in labels/val


In [22]:
directory = "../cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc"

for root, dirs, files in os.walk(directory):
    level = root.replace(directory, "").count(os.sep)
    indent = " " * (level * 4)
    num_files = len(files)
    print(f"{indent}{os.path.basename(root)}   {num_files}/")
    sub_indent = " " * ((level + 1) * 4)
    #for f in files:
        #print(f"{sub_indent}{f}")

cvat_0209_trainvaltest_processed_unzipped_wonyc   1/
    images   0/
        trainval   0/
        train   1112/
        test   167/
        val   192/
    labels   3/
        trainval   0/
        train   1112/
        test   167/
        val   192/
    .ipynb_checkpoints   1/


### Change .yaml files if needed

In [30]:
# always start the path from the "datasets_dir": "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export"
# path: ./export_0122_0921_processed_unzipped
# train: ./images/train  
# val: ./images/val  

base_dir = "../cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc"

# Filepath to your YAML file
yaml_file = os.path.join(base_dir, "data_nc_5.yaml")

with open(yaml_file, "r") as file:
    data = yaml.safe_load(file)
    print(data)
    data["path"] = "./cvat_0209_trainvaltest_processed_unzipped_wonyc"  # very important!
    print(data["path"])

# Write the changes back to the YAML file
with open(yaml_file, "w") as file:
    yaml.dump(data, file, default_flow_style=False)

print("Updated .yaml path")

{'names': {0: 'prohibAnyT', 1: 'prohibSomeT', 2: 'prohibBusStop', 3: 'prohibClean', 4: 'allow'}, 'nc': 5, 'path': './cvat_0209_trainvaltest_processed_unzipped', 'test': './images/test', 'train': './images/train', 'val': './images/val'}
./cvat_0209_trainvaltest_processed_unzipped_wonyc
Updated .yaml path


# Train model_nc_5 on NYC only

In [31]:
# Restart the kernel
torch.cuda.empty_cache()
!nvidia-smi --query-gpu=memory.free --format=csv,nounits

memory.free [MiB]
39202


## Reorganize the label folder

In [32]:
base_dir = "../cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc"

In [33]:
for root, dirs, files in os.walk(base_dir):
    level = root.replace(base_dir, "").count(os.sep)
    indent = " " * (level * 4)
    num_files = len(files)
    print(f"{indent}{os.path.basename(root)}   {num_files}/")
    sub_indent = " " * ((level + 1) * 4)
    #for f in files:
        #print(f"{sub_indent}{f}")

cvat_0209_trainvaltest_processed_unzipped_wonyc   1/
    images   0/
        trainval   0/
        train   1112/
        test   167/
        val   192/
    labels   3/
        trainval   0/
        train   1112/
        test   167/
        val   192/
    .ipynb_checkpoints   1/


## Train by freeze_0_10 (with 23rd DFL frozen)

In [34]:
model_nc_5_nyc = YOLO('yolo11l.pt')

In [35]:
# where is .yaml file?
yaml_file = os.path.join(base_dir, "data_nc_5.yaml")
yaml_file

'../cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc/data_nc_5.yaml'

In [36]:
model_nc_5_nyc.train(data=yaml_file, freeze=11, epochs=30, batch=32, imgsz=640, workers=4,
                 name="model_nc_5_0209_l_30_nyc")

New https://pypi.org/project/ultralytics/8.3.74 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=../cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc/data_nc_5.yaml, epochs=30, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=model_nc_5_0209_l_30_nyc, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_

train: Scanning /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc/labels/train... 1112 images, 254 backgrounds, 0 corrupt: 100%|██████████| 1112/1112 [00:00<00:00, 2710.79it/s]

train: New cache created: /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc/labels/train.cache



val: Scanning /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc/labels/val... 192 images, 50 backgrounds, 0 corrupt: 100%|██████████| 192/192 [00:00<00:00, 1558.19it/s]

val: New cache created: /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc/labels/val.cache


...

                   all        192        162      0.897       0.77       0.89      0.704



30 epochs completed in 0.094 hours.
Optimizer stripped from /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nyc/weights/last.pt, 51.2MB
Optimizer stripped from /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nyc/weights/best.pt, 51.2MB

Validating /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nyc/weights/best.pt...
Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
YOLO11l summary (fused): 464 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.60it/s]


                   all        192        162      0.867      0.805       0.89      0.707
            prohibAnyT         57         57      0.934      0.877      0.966      0.769
           prohibSomeT         48         49      0.828      0.918      0.907      0.711
         prohibBusStop         27         27      0.813      0.643       0.76      0.568
           prohibClean         21         21      0.946      0.835      0.961      0.746
                 allow          8          8      0.817       0.75      0.857       0.74
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nyc


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x15539c7f3130>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

## Validate

In [37]:
model_nc_5_nyc.val(data=yaml_file, imgsz=640, line_width=10,
                 name="model_nc_5_0209_l_30_nyc_val")

Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
YOLO11l summary (fused): 464 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs


val: Scanning /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0209_trainvaltest_processed_unzipped_wonyc/labels/val.cache... 192 images, 50 backgrounds, 0 corrupt: 100%|██████████| 192/192 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.52it/s]


                   all        192        162      0.867      0.804      0.891      0.706
            prohibAnyT         57         57      0.935      0.877      0.967      0.766
           prohibSomeT         48         49      0.827      0.918      0.909      0.717
         prohibBusStop         27         27      0.811      0.637       0.76      0.559
           prohibClean         21         21      0.946      0.835      0.961      0.742
                 allow          8          8      0.817       0.75      0.857      0.746
Speed: 0.2ms preprocess, 5.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nyc_val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x15539eaf7340>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

# Train model_nc_5 on NYC & ATL

In [19]:
# Restart the kernel
torch.cuda.empty_cache()
!nvidia-smi --query-gpu=memory.free --format=csv,nounits

memory.free [MiB]
40443


## Reorganize the label folder

In [4]:
base_dir = "../cvat_export/cvat_0209_trainvaltest_processed_unzipped"

In [5]:
for root, dirs, files in os.walk(base_dir):
    level = root.replace(base_dir, "").count(os.sep)
    indent = " " * (level * 4)
    num_files = len(files)
    print(f"{indent}{os.path.basename(root)}   {num_files}/")
    sub_indent = " " * ((level + 1) * 4)
    #for f in files:
        #print(f"{sub_indent}{f}")

cvat_0209_trainvaltest_processed_unzipped   1/
    images   0/
        trainval   0/
        train   1346/
        test   167/
        val   238/
    labels   3/
        trainval   0/
        train   1346/
        test   167/
        val   238/
    .ipynb_checkpoints   1/


## Train by freeze_0_10 (with 23rd DFL frozen)

In [22]:
model_nc_5_nycatl = YOLO('yolo11l.pt')

In [23]:
# where is .yaml file?
yaml_file = os.path.join(base_dir, "data_nc_5.yaml")
yaml_file

'../cvat_export/cvat_0209_trainvaltest_processed_unzipped/data_nc_5.yaml'

In [24]:
model_nc_5_nycatl.train(data=yaml_file, freeze=11, epochs=30, batch=32, imgsz=640, workers=4,
                 name="model_nc_5_0209_l_30_nycatl")

New https://pypi.org/project/ultralytics/8.3.74 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=../cvat_export/cvat_0209_trainvaltest_processed_unzipped/data_nc_5.yaml, epochs=30, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=model_nc_5_0209_l_30_nycatl, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms

train: Scanning /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0209_trainvaltest_processed_unzipped/labels/train.cache... 1346 images, 339 backgrounds, 0 corrupt: 100%|██████████| 1346/1346 [00:00<?, ?it/s]
val: Scanning /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0209_trainvaltest_processed_unzipped/labels/val.cache... 238 images, 74 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]
/home/hice1/jlee4338/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Plotting labels to /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nycatl/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nycatl
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30      8.35G      1.049      3.814     0.9956          0        640: 100%|██████████| 43/43 [00:12<00:00,  3.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.79it/s]

                   all        238        189   0.000134     0.0855   7.76e-05   5.64e-05


...

                   all        238        189       0.94      0.762      0.877      0.681



30 epochs completed in 0.113 hours.
Optimizer stripped from /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nycatl/weights/last.pt, 51.2MB
Optimizer stripped from /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nycatl/weights/best.pt, 51.2MB

Validating /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nycatl/weights/best.pt...
Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
YOLO11l summary (fused): 464 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.85it/s]


                   all        238        189      0.919      0.765      0.885      0.687
            prohibAnyT         73         74      0.966      0.824      0.955      0.761
           prohibSomeT         52         53      0.841      0.798        0.9      0.703
         prohibBusStop         27         27      0.932       0.63      0.775      0.567
           prohibClean         21         21      0.953      0.857      0.962      0.736
                 allow         13         14      0.902      0.714      0.832      0.668
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nycatl


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x1553a11d6f50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

## Validate

In [25]:
model_nc_5_nycatl.val(data=yaml_file, imgsz=640, line_width=10,
                 name="model_nc_5_0209_l_30_nycatl_val")

Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
YOLO11l summary (fused): 464 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs


val: Scanning /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0209_trainvaltest_processed_unzipped/labels/val.cache... 238 images, 74 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.14it/s]


                   all        238        189      0.925      0.762      0.885      0.688
            prohibAnyT         73         74      0.983      0.824      0.955      0.762
           prohibSomeT         52         53      0.838      0.783        0.9      0.704
         prohibBusStop         27         27      0.937       0.63      0.775      0.555
           prohibClean         21         21       0.96      0.857      0.962      0.749
                 allow         13         14      0.908      0.714      0.832      0.668
Speed: 0.1ms preprocess, 5.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_5_0209_l_30_nycatl2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x1553a0985e70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        